#  Walt Disney World On Site Hotel Addresses

### Using touringplans.com website, scrape 3 web pages, put information into a dataframe, and save to csv file.  

#### The first web page contains resort hotel addresses, the second web page, which will be used as the master list of resort hotels, contains the resort area information, and the third web page contains the number of rooms for each resort hotel.  After scraping, information for each web page is put into a dataframe.

#### The 3 dataframes will be combined into one dataframe and saved to a csv file.

Import libraries.

In [2]:
import numpy as np
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from unidecode import unidecode
from fuzzywuzzy import fuzz, process

Define function to get table from soup object.

In [166]:
def get_table_from_soup(soup, columns, drop_rows):
    l = []
    table_rows=soup.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        row = [unidecode(d.text.strip()) for d in td]
        l.append(row)
    df=pd.DataFrame(l)
    df=df.drop([2, 3], axis=1)
    df=df.drop(drop_rows, axis=0)
    df.columns=columns
    return df

### 1.  Get addresses from touringplans.com page.

In [167]:
url='https://touringplans.com/walt-disney-world/hotels/addresses'
response = requests.get(url)
response

<Response [200]>

Use BeautifulSoup to parse web page.

In [168]:
soup = BeautifulSoup(response.text, 'html.parser')

Get rows from web page.

In [169]:
columns=['Resort', 'Address']
drop_rows=[41, 42]
df=get_table_from_soup(soup, columns, drop_rows)
df['Address']=df['Address'].map(lambda a: a.replace('\t', '').replace('\n', ' '))
df

,Resort,Address
0,B Resort & Spa - Disney Springs Resort Area,"1905 Hotel Plaza Boulevard Lake Buena Vista, F..."
1,Bay Lake Tower at Disney's Contemporary Resort,"4600 North World Drive Lake Buena Vista, FL 32830"
2,Best Western Lake Buena Vista - Disney Springs...,"2000 Hotel Plaza Boulevard Lake Buena Vista, F..."
3,Boulder Ridge Villas at Disney's Wilderness Lodge,"901 Timberline Drive Orlando, FL 32830"
4,Copper Creek Villas & Cabins at Disney's Wilde...,"901 Timberline Drive Orlando, FL 32830"
5,Disney's All-Star Movies Resort,"1901 West Buena Vista Drive Lake Buena Vista, ..."
6,Disney's All-Star Music Resort,"1801 West Buena Vista Drive Lake Buena Vista, ..."
7,Disney's All-Star Sports Resort,"1701 West Buena Vista Drive Orlando, FL 32830"
8,Disney's Animal Kingdom Lodge - Jambo House,"2901 Osceola Parkway Bay Lake, FL 32830"
9,Disney's Animal Kingdom Villas - Kidani Village,"2901 Osceola Parkway Lake Buena Vista, FL 32830"


### 2.  Get resort area for hotel.  This will be the master list of resort hotels.

In [193]:
url='https://touringplans.com/walt-disney-world/hotels'
response = requests.get(url)
response

<Response [200]>

In [194]:
soup = BeautifulSoup(response.text, 'html.parser')
soup.find_all('h2')

[<h2>Value Resort Hotels</h2>,
 <h2>Moderate Resort Hotels</h2>,
 <h2>Deluxe Resort Hotels</h2>,
 <h2>Deluxe Villas</h2>,
 <h2>Magic Kingdom Area Resorts</h2>,
 <h2>Epcot Area Resorts</h2>,
 <h2>Animal Kingdom Area Resorts</h2>,
 <h2>Bonnet Creek Area Resorts</h2>]

In [172]:
print(soup.find('h2',text='Magic Kingdom Area Resorts').find_next('li'))

<li><a href="/walt-disney-world/hotels/copper-creek-villas">Copper Creek Villas &amp; Cabins at Disney's Wilderness Lodge</a> (Deluxe Villa)</li>


In [173]:
start = soup.find('h2',text='Magic Kingdom Area Resorts')
start.next_element
for element in start.next_elements:
    print(repr(element))

'Magic Kingdom Area Resorts'
'\n'
<br/>
'\n'
<ul>
<li><a href="/walt-disney-world/hotels/copper-creek-villas">Copper Creek Villas &amp; Cabins at Disney's Wilderness Lodge</a> (Deluxe Villa)</li>
<li><a href="/walt-disney-world/hotels/disneys-contemporary-resort">Disney’s Contemporary Resort</a> (Deluxe Resort)</li>
<li><a href="/walt-disney-world/hotels/disneys-grand-floridian-resort">Disney’s Grand Floridian Resort</a> (Deluxe Resort)</li>
<li><a href="/walt-disney-world/hotels/disneys-polynesian-resort">Disney’s Polynesian Resort</a> (Deluxe Resort)</li>
<li><a href="/walt-disney-world/hotels/disneys-wilderness-lodge">Disney’s Wilderness Lodge</a> (Deluxe Resort)</li>
<li><a href="/walt-disney-world/hotels/bay-lake-tower-at-disneys-contemporary-resort">Bay Lake Tower at Disney’s Contemporary Resort</a> (Deluxe Villa)</li>
<li><a href="/walt-disney-world/hotels/the-villas-at-disneys-grand-floridian-resort-and-spa">The Villas at Disney’s Grand Floridian Resort &amp; Spa</a> (Deluxe Vi

<a href="/walt-disney-world/hotels/marinas">Marinas</a>
'Marinas'
<br/>
'\n'
<a href="/walt-disney-world/hotels/beaches">Beaches</a>
'Beaches'
<br/>
'\n'
<a href="/walt-disney-world/hotels/tennis">Tennis</a>
'Tennis'
<br/>
'\n'
<a href="/walt-disney-world/hotels/biking">Biking</a>
'Biking'
<br/>
'\n'
<a href="/walt-disney-world/hotels/suites">Suites</a>
'Suites'
<br/>
'\n'
<a href="/walt-disney-world/hotels/concierge-floors">Concierge Floors</a>
'Concierge Floors'
<br/>
'\n'
<a href="/walt-disney-world/hotels/room-service">Room Service</a>
'Room Service'
<br/>
'\n'
<a href="/walt-disney-world/hotels/wired-internet">Wired Internet</a>
'Wired Internet'
<br/>
'\n'
<a href="/walt-disney-world/hotels/wireless-internet">Wireless Internet</a>
'Wireless Internet'
<br/>
'\n'
<a href="/walt-disney-world/hotels/shuttle-to-parks">Shuttle to Parks</a>
'Shuttle to Parks'
<br/>
'\n'
<a href="/walt-disney-world/hotels/airport-transportation">Airport Transportation</a>
'Airport Transportation'
<br/>
'\

<a href="/hotel_maps">Hotel Room Views</a>
'Hotel Room Views'
'\n'
<li>
<a href="/walt-disney-world/disney-travel-packages">Travel Packages</a>
</li>
'\n'
<a href="/walt-disney-world/disney-travel-packages">Travel Packages</a>
'Travel Packages'
'\n'
'\n'
<li><a href="/walt-disney-world/staying-onsite-or-offsite">Onsite or Offsite</a></li>
<a href="/walt-disney-world/staying-onsite-or-offsite">Onsite or Offsite</a>
'Onsite or Offsite'
'\n'
<li><a href="/walt-disney-world/resort-benefits">Resort Benefits</a></li>
<a href="/walt-disney-world/resort-benefits">Resort Benefits</a>
'Resort Benefits'
'\n'
<li><a href="/walt-disney-world/best-spas">Best Spas</a></li>
<a href="/walt-disney-world/best-spas">Best Spas</a>
'Best Spas'
'\n'
<li><a href="/walt-disney-world/resort-dining">Resort Dining</a></li>
<a href="/walt-disney-world/resort-dining">Resort Dining</a>
'Resort Dining'
'\n'
'\n'
'\n'
<br/>
'\n'
<p><a href="/walt-disney-world/dining">Disney World Dining</a></p>
<a href="/walt-disney-w

</div>
'\n'
<div class="fb-like-box" data-colorscheme="light" data-header="false" data-href="https://www.facebook.com/TouringPlans" data-show-border="false" data-show-faces="true" data-stream="false"></div>
'\n'
'\n'
'END ONE-THIRD LAST'
'\n'
'END SECTION'
'\n'
<div class="section alternate">
<div class="holder">
<div class="one text-align-center color">
<p>
<a href="https://touringplans.com/blog/" rel="nofollow" target="_blank"><img alt="Blog" border="0" height="27" src="//cdn0.touringplans.com/assets/tp_blogfooter-5805e1f0547ae1cf0d541c9fa34547150d7c61089a811fc694b9dd9ab5bbfcff.gif" width="100"/></a>
<a href="https://twitter.com/TouringPlans" rel="nofollow" target="_blank"><img alt="Twitter" border="0" height="19" src="//cdn2.touringplans.com/assets/twitter-503d06283b8330e1b18bbca372a472d324967450db983baf626e7d123d0a6253.gif" style="padding-bottom: 6px" width="100"/></a>
<a href="https://facebook.com/TouringPlans" rel="nofollow" target="_blank"><img alt="Facebook" border="0" height="

<div id="facebookholder">
<div class="fb-share-button" data-type="button"></div>
</div>
'\n'
<div class="fb-share-button" data-type="button"></div>
'\n'
'\n'
'\n'
<a href="#" id="back-top-arrow">Top</a>
'Top'
'\n'
'\n'
' END NEW BACK TOP SECTION '
'\n'
' END NEW BACK TOP SECTION '
'\n'
' END NEW BACK TOP SECTION '
'\n'
' END NEW BACK TOP SECTION '
'\n'
' Google Code for Remarketing Tag '
'\n'
'------------------------------------------------\nRemarketing tags may not be associated with personally identifiable information or placed on pages related to sensitive categories. See more information and instructions on how to setup the tag on: http://google.com/ads/remarketingsetup\n-------------------------------------------------'
'\n'
<script type="text/javascript">
/* <![CDATA[ */
var google_conversion_id = 880380104;
var google_custom_params = window.google_tag_params;
var google_remarketing_only = true;
/* ]]> */
</script>
'\n/* <![CDATA[ */\nvar google_conversion_id = 880380104;\nvar g

In [195]:
def get_resort_by_area(soup_area, number_of_resorts, area):
    start = soup.find('h2',text=soup_area)
    resorts=[]
    for i in range(number_of_resorts):
        resort_name=start.find_next('li').text
        resorts.append(resort_name)
        start=start.find_next('li')
    df=pd.DataFrame(resorts, columns=['Resort'])
    df['Resort Area']=area
    return df

In [196]:
df_mk_resorts=get_resort_by_area('Magic Kingdom Area Resorts', 9, 'Magic Kingdom')
df_mk_resorts

,Resort,Resort Area
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom
1,Disney’s Contemporary Resort (Deluxe Resort),Magic Kingdom
2,Disney’s Grand Floridian Resort (Deluxe Resort),Magic Kingdom
3,Disney’s Polynesian Resort (Deluxe Resort),Magic Kingdom
4,Disney’s Wilderness Lodge (Deluxe Resort),Magic Kingdom
5,Bay Lake Tower at Disney’s Contemporary Resort...,Magic Kingdom
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom
7,Disney’s Wilderness Lodge Villas (Deluxe Villa),Magic Kingdom
8,Disney’s Fort Wilderness Resort Cabins (Modera...,Magic Kingdom


In [197]:
df_e_resorts=get_resort_by_area('Epcot Area Resorts', 11, 'Epcot')
df_e_resorts

,Resort,Resort Area
0,Disney’s Beach Club Resort (Deluxe Resort),Epcot
1,Disney’s BoardWalk Inn (Deluxe Resort),Epcot
2,Disney’s Yacht Club Resort (Deluxe Resort),Epcot
3,Dolphin at Walt Disney World (Not operated by ...,Epcot
4,Swan at Walt Disney World (Not operated by Di...,Epcot
5,Disney’s Beach Club Villas (Deluxe Villa),Epcot
6,Disney’s BoardWalk Villas (Deluxe Villa),Epcot
7,Disney's Riviera Resort (Deluxe Villa),Epcot
8,Disney’s Caribbean Beach Resort (Moderate Resort),Epcot
9,Disney’s Art of Animation Resort (Value Resort),Epcot


In [198]:
df_ak_resorts=get_resort_by_area('Animal Kingdom Area Resorts', 6, 'Animal Kingdom')
df_ak_resorts

,Resort,Resort Area
0,Disney’s Animal Kingdom Lodge (Deluxe Resort),Animal Kingdom
1,Disney’s Animal Kingdom Villas (Deluxe Villa),Animal Kingdom
2,Disney’s Coronado Springs Resort (Moderate Res...,Animal Kingdom
3,Disney’s All-Star Movies Resort (Value Resort),Animal Kingdom
4,Disney’s All-Star Music Resort (Value Resort),Animal Kingdom
5,Disney’s All-Star Sports Resort (Value Resort),Animal Kingdom


In [199]:
df_ds_resorts=get_resort_by_area('Bonnet Creek Area Resorts', 4, 'Disney Springs')
df_ds_resorts

,Resort,Resort Area
0,Disney’s Old Key West Resort (Deluxe Villa),Disney Springs
1,Disney’s Saratoga Springs Resort & Spa (Deluxe...,Disney Springs
2,Disney’s Port Orleans Resort – French Quarter ...,Disney Springs
3,Disney’s Port Orleans Resort – Riverside (Mode...,Disney Springs


In [200]:
df_resort_areas=df_mk_resorts.append([df_e_resorts, df_ak_resorts, df_ds_resorts])
df_resort_areas.reset_index(drop=True, inplace=True)
df_resort_areas

,Resort,Resort Area
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom
1,Disney’s Contemporary Resort (Deluxe Resort),Magic Kingdom
2,Disney’s Grand Floridian Resort (Deluxe Resort),Magic Kingdom
3,Disney’s Polynesian Resort (Deluxe Resort),Magic Kingdom
4,Disney’s Wilderness Lodge (Deluxe Resort),Magic Kingdom
5,Bay Lake Tower at Disney’s Contemporary Resort...,Magic Kingdom
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom
7,Disney’s Wilderness Lodge Villas (Deluxe Villa),Magic Kingdom
8,Disney’s Fort Wilderness Resort Cabins (Modera...,Magic Kingdom
9,Disney’s Beach Club Resort (Deluxe Resort),Epcot


In [16]:
'Disney’s Contemporary Resort (Deluxe Resort)'.split('(')[0]

'Disney’s Contemporary Resort '

In [206]:
df_resort_areas['Resort']=df_resort_areas['Resort'].apply(lambda x: x.split('(')[0])
df_resort_areas

,Resort,Resort Area
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom
1,Disney’s Contemporary Resort,Magic Kingdom
2,Disney’s Grand Floridian Resort,Magic Kingdom
3,Disney’s Polynesian Resort,Magic Kingdom
4,Disney’s Wilderness Lodge,Magic Kingdom
5,Bay Lake Tower at Disney’s Contemporary Resort,Magic Kingdom
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom
7,Disney’s Wilderness Lodge Villas,Magic Kingdom
8,Disney’s Fort Wilderness Resort Cabins,Magic Kingdom
9,Disney’s Beach Club Resort,Epcot


In [217]:
def fix_resort_names(name):
    fix_name=name
    if fuzz.token_sort_ratio("Disney's Wilderness Lodge Villas", name)==100:
        fix_name="Boulder Ridge Villas at Disney's Wilderness Lodge"
    if fuzz.token_sort_ratio("Disney's Polynesian Resort", name)==100:
        fix_name="Disney's Polynesian Village Resort"
    if 'Animal Kingdom Lodge' in name:
        fix_name=name+' - Jambo House'
    if 'Animal Kingdom Villas' in name:
        fix_name=name+' - Kidani Village'
    return fix_name
df_resort_areas['Resort']=df_resort_areas['Resort'].map(lambda name: fix_resort_names(name))
df_resort_areas

,Resort,Resort Area
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom
1,Disney’s Contemporary Resort,Magic Kingdom
2,Disney’s Grand Floridian Resort,Magic Kingdom
3,Disney's Polynesian Village Resort,Magic Kingdom
4,Disney’s Wilderness Lodge,Magic Kingdom
5,Bay Lake Tower at Disney’s Contemporary Resort,Magic Kingdom
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom
7,Boulder Ridge Villas at Disney's Wilderness Lodge,Magic Kingdom
8,Disney’s Fort Wilderness Resort Cabins,Magic Kingdom
9,Disney’s Beach Club Resort,Epcot


In [207]:
df['Resort'][27]

'Disney’s Wilderness Lodge '

In [208]:
df_resort_areas['Resort'][4]

'Disney’s Wilderness Lodge '

In [98]:
fuzz.token_sort_ratio(df['Resort'][27], df_resort_areas['Resort'][7])

88

In [80]:
fuzz.token_sort_ratio(df_resort_areas['Resort'][23], df_resort_areas['Resort'][24])

89

In [81]:
print(df_resort_areas['Resort'][23], ' ', df_resort_areas['Resort'][24])

Disney’s All-Star Movies Resort    Disney’s All-Star Music Resort 


In [205]:
print(df['Resort'][20], ' ', df_resort_areas['Resort'][3])
fuzz.token_sort_ratio(df['Resort'][20], df_resort_areas['Resort'][3])

Disney's Polynesian Village Resort   Disney’s Polynesian Resort 


87

### 3.  Get number of room for each hotel.

In [182]:
url='https://touringplans.com/walt-disney-world/hotels/number-rooms'
response = requests.get(url)
response

<Response [200]>

In [183]:
soup = BeautifulSoup(response.text, 'html.parser')
rows=soup.find_all('td')

In [184]:
columns=['Resort', 'Number of Rooms']
drop_rows=[42, 43]
df_rooms=get_table_from_soup(soup, columns, drop_rows)
df_rooms

,Resort,Number of Rooms
0,B Resort & Spa - Disney Springs Resort Area,394
1,Bay Lake Tower at Disney's Contemporary Resort,295
2,Best Western Lake Buena Vista - Disney Springs...,325
3,Boulder Ridge Villas at Disney's Wilderness Lodge,136
4,Copper Creek Villas & Cabins at Disney's Wilde...,220
5,Disney's All-Star Movies Resort,"1,920"
6,Disney's All-Star Music Resort,"1,604"
7,Disney's All-Star Sports Resort,"1,920"
8,Disney's Animal Kingdom Lodge - Jambo House,972
9,Disney's Animal Kingdom Villas - Kidani Village,458


### 4.  Use FuzzyWuzzy to match resort names in each dataframe

In [74]:
print(df_resort_areas['Resort'][10])
len(df_resort_areas)-1

Disney’s BoardWalk Inn 


29

In [75]:
df_resort_areas['Resort'][4]

'Disney’s Wilderness Lodge '

In [76]:
fuzz.token_sort_ratio("Disney's Wilderness Lodge", df_resort_areas['Resort'][4])

100

In [77]:
df['Resort'][3]

"Boulder Ridge Villas at Disney's Wilderness Lodge"

In [209]:
fuzz.token_sort_ratio("Disney's Wilderness Lodge", df_resort_areas['Resort'][0])

62

In [218]:
for i in range(len(df_resort_areas)):
    for j in range(len(df)):
        if fuzz.token_sort_ratio(df['Resort'][j], df_resort_areas['Resort'][i]) > 89:
            df['Resort'][j]=df_resort_areas['Resort'][i]

In [219]:
df

,Resort,Address
0,B Resort & Spa - Disney Springs Resort Area,"1905 Hotel Plaza Boulevard Lake Buena Vista, F..."
1,Bay Lake Tower at Disney’s Contemporary Resort,"4600 North World Drive Lake Buena Vista, FL 32830"
2,Best Western Lake Buena Vista - Disney Springs...,"2000 Hotel Plaza Boulevard Lake Buena Vista, F..."
3,Boulder Ridge Villas at Disney's Wilderness Lodge,"901 Timberline Drive Orlando, FL 32830"
4,Copper Creek Villas & Cabins at Disney's Wilde...,"901 Timberline Drive Orlando, FL 32830"
5,Disney’s All-Star Movies Resort,"1901 West Buena Vista Drive Lake Buena Vista, ..."
6,Disney’s All-Star Music Resort,"1801 West Buena Vista Drive Lake Buena Vista, ..."
7,Disney’s All-Star Sports Resort,"1701 West Buena Vista Drive Orlando, FL 32830"
8,Disney’s Animal Kingdom Lodge - Jambo House,"2901 Osceola Parkway Bay Lake, FL 32830"
9,Disney’s Animal Kingdom Villas - Kidani Village,"2901 Osceola Parkway Lake Buena Vista, FL 32830"


In [187]:
df_resort_areas['Resort'][20]

'Disney’s Animal Kingdom Lodge  - Jambo House'

In [220]:
joined=df_resort_areas.join(df.set_index('Resort'), how='left', on='Resort')
joined

,Resort,Resort Area,Address
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830"
1,Disney’s Contemporary Resort,Magic Kingdom,"4600 North World Drive Orlando, FL 32830"
2,Disney’s Grand Floridian Resort,Magic Kingdom,"4401 Floridian Way Orlando, FL 32830"
3,Disney's Polynesian Village Resort,Magic Kingdom,"1600 Seven Seas Drive Orlando, FL 32830"
4,Disney’s Wilderness Lodge,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830"
5,Bay Lake Tower at Disney’s Contemporary Resort,Magic Kingdom,"4600 North World Drive Lake Buena Vista, FL 32830"
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom,"4401 Floridian Way Orlando, FL 32830"
7,Boulder Ridge Villas at Disney's Wilderness Lodge,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830"
8,Disney’s Fort Wilderness Resort Cabins,Magic Kingdom,"4510 North Fort Wilderness Trail Orlando, FL 3..."
9,Disney’s Beach Club Resort,Epcot,"1800 Epcot Resort Boulevard Lake Buena Vista, ..."


In [221]:
joined['Address'].isnull().sum()

0

In [222]:
joined[joined['Address'].isnull()==True]

,Resort,Resort Area,Address


In [223]:
for i in range(len(joined)):
    for j in range(len(df_rooms)):
        if fuzz.token_sort_ratio(df_rooms['Resort'][j], joined['Resort'][i]) > 89:
            df_rooms['Resort'][j]=joined['Resort'][i]

In [226]:
df_rooms.head()

,Resort,Number of Rooms
0,B Resort & Spa - Disney Springs Resort Area,394
1,Bay Lake Tower at Disney’s Contemporary Resort,295
2,Best Western Lake Buena Vista - Disney Springs...,325
3,Boulder Ridge Villas at Disney's Wilderness Lodge,136
4,Copper Creek Villas & Cabins at Disney's Wilde...,220


In [227]:
joined=joined.join(df_rooms.set_index('Resort'), how='left', on='Resort')
joined

,Resort,Resort Area,Address,Number of Rooms
0,Copper Creek Villas & Cabins at Disney's Wilde...,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830",220
1,Disney’s Contemporary Resort,Magic Kingdom,"4600 North World Drive Orlando, FL 32830",655
2,Disney’s Grand Floridian Resort,Magic Kingdom,"4401 Floridian Way Orlando, FL 32830",867
3,Disney's Polynesian Village Resort,Magic Kingdom,"1600 Seven Seas Drive Orlando, FL 32830",847
4,Disney’s Wilderness Lodge,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830",430
5,Bay Lake Tower at Disney’s Contemporary Resort,Magic Kingdom,"4600 North World Drive Lake Buena Vista, FL 32830",295
6,The Villas at Disney’s Grand Floridian Resort ...,Magic Kingdom,"4401 Floridian Way Orlando, FL 32830",147
7,Boulder Ridge Villas at Disney's Wilderness Lodge,Magic Kingdom,"901 Timberline Drive Orlando, FL 32830",136
8,Disney’s Fort Wilderness Resort Cabins,Magic Kingdom,"4510 North Fort Wilderness Trail Orlando, FL 3...",409
9,Disney’s Beach Club Resort,Epcot,"1800 Epcot Resort Boulevard Lake Buena Vista, ...",583


In [229]:
joined['Number of Rooms'].isnull().sum()

0

In [230]:
joined.to_csv('wdw_resorts_addresses.csv', index=False)